### Большая часть задач требует операций с файлами, поэтому для воспроизводимости результатов работы вашего кода эти файлы нужно создавать

### Можете их всех создавать в текущей папке, быстро и удобно это можно сделать таким способом:

In [1]:
text = """123
4567
 8910
"""
with open('task_1.1_source', 'w') as f:
    f.write(text)

In [2]:
!cat task_1.1_source

123
4567
 8910


# Базовый уровень

## Задание 1.1

Напишите функцию, копирующую файл

source_path должен существовать, а отсутствующие промежуточные папки из target_path должны быть созданы

Подсказка: можно прочитать исходный файл и записать в новый. 

Опциональное усложнение: попробуйте написать так, чтобы функция работала и с большими файлами (нужно, чтобы содежимое файла не считывалось полностью в память)

In [3]:
import os

In [4]:
def create_path_if_needed(target_path):
    target_dir = os.path.dirname(target_path)
    if target_dir != '' and not os.path.exists(target_dir):
        os.makedirs(target_dir)


def copy_file(source_path, target_path):
    create_path_if_needed(target_path)
        
    with open(source_path, 'r') as fi, open(target_path, 'w') as fo:
        fo.write(fi.read())

In [5]:
def copy_file_big(source_path, target_path, block_size=3):
    create_path_if_needed(target_path)
        
    with open(source_path, 'r') as fi, open(target_path, 'w') as fo:
        block = fi.read(block_size)
        while block != '':
            fo.write(block)
            block = fi.read(block_size)

In [6]:
copy_file('task_1.1_source', 'tmp/task_1.1_source')

In [7]:
copy_file_big('task_1.1_source', 'tmp/task_1.1_source')

In [8]:
!ls

1_Intro_hw                      task_1.2_source
2_Syntax_part1_hw.ipynb         task_1.2_target
2_Syntax_part1_hw_done.ipynb    task_2.1_source
3_Syntax_part2_hw.ipynb         task_2.2_source
3_Syntax_part2_hw_done.ipynb    task_2.2_target
4_Encodings_Files_HW.ipynb      task_3.2_source_1
4_Encodings_Files_HW_done.ipynb task_3.2_source_2
5_Pandas_part1_hw               task_3.2_target
task_1.1_source                 tmp


In [9]:
!ls tmp

task_1.1_source


In [10]:
cat tmp/task_1.1_source

123
4567
 8910


## Задание 1.2

##### В файле source_path находится англо-латинский словарь, то есть список слов на английском языке и их переводы на латинский язык (переводов может быть несколько). Необходимо создать из него латино-английский словарь по пути target_path. Например, из словаря



apple - malum, pomum, popula

fruit - baca, bacca, popum

punishment - malum, multa



##### нужно сделать словарь.

baca - fruit

bacca - fruit

malum - apple, punishment

multa - punishment

pomum - apple

popula - apple

popum - fruit

##### Не забудьте, что слова нужно расположить в алфавитном порядке.
##### Будет полезна функция strip у строки для удаления пробелов


In [11]:
from collections import defaultdict
import re


def dump_dict_2_file(words_dict, target_path, encoding='utf-8'):
    create_path_if_needed(target_path)
    with open(target_path, 'w')as fo:
        for word, translations in sorted(words_dict.items()):
            fo.write('{} - {}\n'.format(word.encode(encoding), u', '.join(translations).encode(encoding)))


def eng_lat_2_lat_eng(source_path, target_path):
    lat_eng_dict = defaultdict(list)
    with open(source_path, 'r') as fi:
        for line in fi:
            tokens = re.findall('[a-zA-Z]+', line)
            word, translations = tokens[0], tokens[1:]
            for translation in translations:
                lat_eng_dict[translation].append(word)
    dump_dict_2_file(lat_eng_dict, target_path)
            

In [12]:
text = """apple - malum, pomum, popula
fruit - baca, bacca, popum
punishment - malum, multa
"""
with open('task_1.2_source', 'w') as f:
    f.write(text)

In [13]:
eng_lat_2_lat_eng('task_1.2_source', 'task_1.2_target')

In [14]:
!cat task_1.2_target

baca - fruit
bacca - fruit
malum - apple, punishment
multa - punishment
pomum - apple
popula - apple
popum - fruit


## Задание 1.3

Напишите функцию, вычисляющую длину русского слова, независимо от его кодировки

Если параметр encoding задан, то нужно декодировать текст при помощи этой кодировки. А если он не задан, то кодировку надо определить самостоятельно.

##### Если кодировка не задана, то точное решение здесь написать не получится. Но можно воспользоваться функцией my_decoder из лекции

In [15]:
import chardet

def my_len(word, encoding=None):
    if type(word) is not unicode:
        if encoding is None:
            encoding = chardet.detect(word)['encoding']
        word = word.decode(encoding)
    return len(word)

In [16]:
my_len('текст') == 5

True

In [17]:
my_len(u'текст') == 5

True

In [18]:
my_len(u'ааааа'.encode('utf_16'), encoding='utf_16') == 5

True

In [19]:
my_len(u'текст'.encode('windows-1251')) == 5

True

# Продвинутый уровень

## Задание 2.1

Напишите функцию, которая выписывает список файлов в данной директории и сортирует их в соответствии с их размером. Функция должна получать путь к директории в качестве аргумента и печатать на экран имена всех файлов в ней и их размеры, причем первыми должны идти файлы с наибольшими размерами, а в случае одинакового размер файлы сортируются по алфавиту.

##### Указание. Изучите функции listdir и stat из модуля os и функции isfile и join из модуля os.path.


In [20]:
def get_dir_content_info(dir_path):
    files_with_sizes = [
        (filename, os.stat(os.path.join(dir_path, filename)).st_size)
        for filename in os.listdir(dir_path)
        if os.path.isfile(os.path.join(dir_path, filename))
    ]
    for filename, size in sorted(files_with_sizes, key=lambda (name, size): (-size, name)):
        print filename, size

In [21]:
text = """123
4567
 8910
"""
with open('task_2.1_source', 'w') as f:
    f.write(text)

In [22]:
get_dir_content_info('.')

3_Syntax_part2_hw_done.ipynb 40686
4_Encodings_Files_HW_done.ipynb 27724
3_Syntax_part2_hw.ipynb 17216
2_Syntax_part1_hw_done.ipynb 16536
4_Encodings_Files_HW.ipynb 11514
2_Syntax_part1_hw.ipynb 4853
task_1.2_target 115
task_1.2_source 82
task_3.2_source_1 52
task_3.2_source_2 52
task_2.2_source 48
task_2.2_target 39
task_3.2_target 25
task_1.1_source 15
task_2.1_source 15
.gitignore 1


## Задание 2.2

Аналогично 1.2, но только теперь англо-русский в русско-английский словарь. Дополнительно теперь будет ещё параметр encoding для кодировки файлов. Если он None, то нужно самостоятельно определить кодировку файла.

##### Решение будет почти дублировать 1.2, но сначала текст нужно привести в unicode

In [23]:
def eng_rus_2_rus_eng(source_path, target_path, encoding=None):
    if encoding is None:
        with open(source_path, 'r') as fi:
            encoding = chardet.detect(fi.read())['encoding']
    
    rus_eng_dict = defaultdict(list)
    with open(source_path, 'r') as fi:
        for line in fi:
            tokens = re.findall(u'[а-яА-Яa-zA-Z]+', line.decode(encoding))
            word, translations = tokens[0], tokens[1:]
            for translation in translations:
                rus_eng_dict[translation].append(word)
    dump_dict_2_file(rus_eng_dict, target_path, encoding)

In [24]:
text = """apple - яблоко
mac - мак, яблоко
"""
with open('task_2.2_source', 'w') as f:
    f.write(text)

In [25]:
eng_rus_2_rus_eng('task_2.2_source', 'task_2.2_target', 'utf-8')

In [26]:
!cat task_2.2_target

мак - mac
яблоко - apple, mac


# Сложный уровень

## Задание 3.1

Существует исследование, говорящее о том, что в словах текста можно произвольно переставить буквы (не затрагивая первую и последнюю), и от этого читабельность текста практически не ухудшится. Напишите функцию для проверки этого факта. Ваша функция должна получать на вход какой-нибудь текст (может быть и на русском языке) и переставлять буквы в его словах случайным образом. 

Если seed None, то буквы переставляются не случайным образом, а сортируются по алфавиту (латинские символы считаем меньше кириллических). Если seed не None, то нужно использовать это seed при инициализации рандома. Сравните результаты.

##### будем считать, что текст это последовательность кириллических и латинских символов разделённая пробелом
##### вам потребуется модуль random и функции random.seed и random.shuffle
##### проверьте результат работы на не юникодовских строчках и объясните результат

In [27]:
import random

def create_word_processor(seed):
    if seed is not None:
        random_gen = random.Random(seed)
        def fun(word):
            if len(word) > 2:
                inner_letters = list(word[1:-1])
                random_gen.shuffle(inner_letters)
                word = word[0] + ''.join(inner_letters) + word[-1]
            return word
    else:
        def fun(word):
            if len(word) > 2:
                inner_letters = list(word[1:-1])
                word = word[0] + ''.join(sorted(inner_letters)) + word[-1]
            return word
    return fun

def text_shuffle(text, seed=None):
    word_processor = create_word_processor(seed)
    print ' '.join(map(word_processor, text.split(' ')))

In [28]:
text_shuffle('вам потребуется модуль random и функции random.seed и random.shuffle')

а��м Ѐ������������������я Ѓ��������ь radnom и у����������Ѹ r.adeemnosd и r.adffhlmnosue


In [29]:
text_shuffle(u'вам потребуется модуль random и функции random.seed и random.shuffle')

вам пбееорсттуя мдлоуь radnom и фикнуци r.adeemnosd и r.adffhlmnosue


In [30]:
text_shuffle('вам потребуется модуль random и функции random.seed и random.shuffle', seed=42)

��в�� е�ѱ����т������уЂ� д���Ѽ���Ќ raondm и �у�Є�иІ��� r.eemsonadd и raoffmdu.nshle


In [31]:
text_shuffle(u'вам потребуется модуль random и функции random.seed и random.shuffle', seed=42)

вам пстбреетоуя млуодь rnoadm и финукци rsodenm.aed и radhfmofs.ulne


In [32]:
text_shuffle(u'вам потребуется модуль random и функции random.seed и random.shuffle', seed=42)

вам пстбреетоуя млуодь rnoadm и финукци rsodenm.aed и radhfmofs.ulne


In [33]:
text_shuffle(u'вам потребуется модуль random и функции random.seed и random.shuffle', seed=47)

вам пеботуртсея мдулоь radnom и фкунции r.oeenasdmd и romh.nsfaludfe


## Задание 3.2

Напишите функцию, которая читает два файла построчно (то есть не загружая всё содержимое в память), и записывает в выходной файл результат лексиграфического сравнение срочки из первого файла и из второго (First, Second и Equal по аналогии c предыдущим ДЗ).

In [34]:
import itertools


def str_compare(fst, snd):
    if fst < snd:
        return 'First'
    elif fst > snd:
        return 'Second'
    else:
        return 'Equal'


def process(fst_source_path, snd_source_path, target_path):
    create_path_if_needed(target_path)
    with open(fst_source_path, 'r') as fi_fst, open(snd_source_path, 'r') as fi_snd, open(target_path, 'w') as fo:
        for fst, snd in itertools.izip(fi_fst, fi_snd):
            fo.write('{}\n'.format(str_compare(fst.strip(), snd.strip())))

In [35]:
text = """apple - яблоко
mac - мак, яблоко
"""
text2 ="""3
4
"""
text3 ="""4
3
"""
with open('task_3.2_source_1', 'w') as f:
    f.write(text)
    f.write(text2)
    
with open('task_3.2_source_2', 'w') as f:
    f.write(text)
    f.write(text3)

In [36]:
process('task_3.2_source_1', 'task_3.2_source_2', 'task_3.2_target')

In [37]:
!cat task_3.2_target

Equal
Equal
First
Second
